## Imports

In [27]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [28]:
df = pd.read_csv('KK_mortgage_map - Sheet1.csv')

## Google Maps API Geolocater Setup

In [29]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [30]:
df['geo_address'] = df['full_address'] + ' Miami-Dade, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [31]:
df.columns

Index(['full_address', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [32]:
df

,full_address,geo_address,loc,point,lat,lon,altitude
0,2308 Northwest Fifth Avenue,"2308 Northwest Fifth Avenue Miami-Dade, FL","(2308 NW 5 Ave, Miami, FL 33127, USA, (25.7992...","(25.7992793, -80.20343729999999, 0.0)",25.799279,-80.203437,0.0
1,2300 Northwest Fifth Avenue,"2300 Northwest Fifth Avenue Miami-Dade, FL","(2300 NW 5 Ave, Miami, FL 33127, USA, (25.7992...","(25.7992418, -80.2033776, 0.0)",25.799242,-80.203378,0.0
2,2380 Northwest Fifth Avenue,"2380 Northwest Fifth Avenue Miami-Dade, FL","(2380 NW 5 Ave, Miami, FL 33127, USA, (25.7997...","(25.799798, -80.2032713, 0.0)",25.799798,-80.203271,0.0
3,508 Northwest 23rd Street,"508 Northwest 23rd Street Miami-Dade, FL","(508 NW 23rd St, Miami, FL 33127, USA, (25.798...","(25.798875, -80.20336999999999, 0.0)",25.798875,-80.203370,0.0
4,514 Northwest 23rd Street,"514 Northwest 23rd Street Miami-Dade, FL","(514 NW 23rd St, Miami, FL 33127, USA, (25.798...","(25.798824, -80.203464, 0.0)",25.798824,-80.203464,0.0
5,518 Northwest 23rd Street,"518 Northwest 23rd Street Miami-Dade, FL","(518 NW 23rd St, Miami, FL 33127, USA, (25.798...","(25.798818, -80.20355599999999, 0.0)",25.798818,-80.203556,0.0
6,2320 Northwest Fifth Avenue,"2320 Northwest Fifth Avenue Miami-Dade, FL","(2320 NW 5 Ave, Miami, FL 33127, USA, (25.7993...","(25.7993737, -80.2034809, 0.0)",25.799374,-80.203481,0.0
7,2324 Northwest Fifth Avenue,"2324 Northwest Fifth Avenue Miami-Dade, FL","(2324 NW 5 Ave, Miami, FL 33127, USA, (25.7995...","(25.799511, -80.2034809, 0.0)",25.799511,-80.203481,0.0
8,2330 Northwest Fifth Avenue,"2330 Northwest Fifth Avenue Miami-Dade, FL","(2330 NW 5 Ave, Miami, FL 33127, USA, (25.7996...","(25.799624, -80.203457, 0.0)",25.799624,-80.203457,0.0
9,490 Northwest 23rd Street,"490 Northwest 23rd Street Miami-Dade, FL","(490 NW 23rd St, Homestead, FL 33030, USA, (25...","(25.7990236, -80.2026759, 0.0)",25.799024,-80.202676,0.0


## Correction section

In [33]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [34]:
df.columns

Index(['full_address', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [35]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [36]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [42]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Mana's Wynwood properties backing $275M credit line")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [41]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [40]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miami-dade_mortgage_properties_map
